In [105]:
import csv
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from numpy import nan
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import roc_auc_score
import xgboost as xgb
plt.style.use('classic')
%matplotlib inline
pd.set_option('display.max_rows', None)

In [106]:
df_train1 = pd.read_csv('sample_data/X_trainData_1.csv')#the datasets must be uploaded in the sample_data directory of google colab to work
df_train2 = pd.read_csv('sample_data/Y_trainData_1.csv')
df_test = pd.read_csv('sample_data/X_testData_1.csv')
df_test = df_test.loc[:, df_test.columns != 'ID']

In [107]:

print(df_train1.shape)
print(df_train2.shape)
print(df_test.shape)

(3393, 222)
(3393, 11)
(1000, 222)


In [108]:
res = pd.concat([df_train1, df_train2], axis=1)
res = res[res.INH != -1]
res2= res[res.INH==1]
y = res['INH']
X = res.drop(['INH'], axis = 1) 
X = X.drop(['OFLX'], axis = 1) 
X = X.drop(['RIF'], axis = 1) 
X = X.drop(['CIP'], axis = 1) 
X = X.drop(['PZA'], axis = 1) 
X = X.drop(['EMB'], axis = 1)
X = X.drop(['CAP'], axis = 1)
X = X.drop(['STR'], axis = 1)
X = X.drop(['AMK'], axis = 1)
X = X.drop(['MOXI'], axis = 1)  
X = X.drop(['KAN'], axis = 1)    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2)

In [109]:
from sklearn.model_selection import GridSearchCV 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

# # Use the random grid to search for best hyperparameters

# # Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 50)]
# # Number of features to consider at every split
# max_features = ['auto', 'sqrt']
# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)
# # Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]
# # Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]
# # Method of selecting samples for training each tree
# bootstrap = [True, False]
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}
# # First create the base model to tune
# rf = RandomForestRegressor()
# # Random search of parameters, using 3 fold cross validation, 
# # search across 100 different combinations, and use all available cores
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 4, verbose=2, random_state=42, n_jobs = -1)
# # Fit the random search model
# rf_random.fit(X, y)
# print(rf_random.best_params_) 
model = RandomForestClassifier(n_estimators=1200, criterion='entropy', max_depth=110, max_features= 'sqrt', min_samples_leaf= 2, min_samples_split=5, bootstrap=True, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict_proba(df_test)
df = pd.read_csv('sample_data/Y_testData_1_nolabels_INH.csv')
ID = df['ID'].tolist()
print(y_pred.shape)
# print(type(y_pred))
# print(len(y_pred))

(1000, 2)


In [110]:
li_id = []
li_val = []
for i in ID:
    li_id.append(i)
    li_val.append((y_pred[i-1][1]))
csv = pd.DataFrame({'ID': li_id, 'INH' :li_val})
csv.to_csv('INH-submit.csv', index=False, header=True)    
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))  
print(roc_auc_score(y_test, y_pred))

[[298  16]
 [ 15 343]]
              precision    recall  f1-score   support

           0       0.95      0.95      0.95       314
           1       0.96      0.96      0.96       358

    accuracy                           0.95       672
   macro avg       0.95      0.95      0.95       672
weighted avg       0.95      0.95      0.95       672

0.9535725723232396
